# Add 1, 4, 7

In [3]:
import glob
import pandas as pd
import re
import os
import sqlite3
import numpy as np

daumNews_dir = glob.glob('../daum_news_dataset/addDB/*.csv')


def preprocessing_articles(data):
    contents_new = []
    for content in data['content']:
        content = str(content).split('\n')
        content = list(filter(lambda s: len(s) > 3, content))
        content_prep = content[:-1]
        content_prep = list(filter(lambda s: len(s) > 30 and \
                                             '▶' not in s and \
                                             '©' not in s and \
                                             '▲' not in s and \
                                             '사진=' not in s and \
                                             '사진제공=' not in s and \
                                             '@' not in s and \
                                             not re.findall("기자 *$", s) and \
                                             not re.findall("제공 *$", s) and \
                                             not re.findall("이하", s) and \
                                             not re.findall("무단전재", s) and \
                                             not re.findall("기자 *= *", s), content_prep))

        content_prep = list(map(lambda s: re.sub("\[.*\]", "", s), content_prep))  # [스포츠서울]
        content_prep = list(map(lambda s: re.sub("\(.*\=.*\)", "", s), content_prep))  # (서울=뉴시스)
        content_prep = list(map(lambda s: re.sub("\【.*\=.*\】", "", s), content_prep))  # 【파이낸셜뉴스 포천=강근주 기자】
        content_prep = list(map(lambda s: re.sub('[^가-힣]', ' ', s), content_prep))  # 한글 제외 영문, 한자, 숫자, 특수문자 모두 제거

        contents_new.append(''.join(content_prep))

    data['news_content'] = contents_new

    return data


if __name__ == "__main__":
    con = sqlite3.connect("../kr_scrapping.db")
    c = con.cursor()
#     c.execute('DROP TABLE IF EXISTS `news_dummy`;')
#     c.execute('''
#             CREATE TABLE `news_dummy` (
#                 `news_id` INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
#                 `news_name` TEXT NOT NULL,
#                 `pubdate` TEXT NOT NULL,
#                 `url` TEXT NOT NULL,
#                 `news_content` TEXT NOT NULL
#             );
#     ''')
#     result_csv = pd.DataFrame()
    for daumNews in daumNews_dir:
        print(daumNews)
        data_prep = pd.DataFrame()
        df = pd.read_csv(daumNews, header=None) #, names=['order', 'news_name', 'pubdate', 'url', 'content'])
        df = df.iloc[:,:5]
        df.columns = ['order', 'news_name', 'pubdate', 'url', 'content']
        df = df.dropna()
        df = df.drop(['order'], axis=1)
        data_prep = preprocessing_articles(df)
        data_prep['news_content'] = data_prep['news_content'].str.strip()
        data_prep['news_content'] = data_prep['news_content'].replace('', np.nan)
        data_prep_drop = data_prep.dropna(axis=0)
        data_prep_drop_dup = data_prep_drop.drop_duplicates(['news_content'], keep='first', inplace=False,
                                                            ignore_index=True)
        result = data_prep_drop_dup[['news_name', 'pubdate', 'url', 'news_content']]
        # result_csv = pd.concat([result_csv, result], axis=0)
        print(len(result))
        result.to_sql('news_dummy', con, if_exists='append', index=False, chunksize=1000)
    # result_csv.to_csv('test.csv', encoding='utf-8-sig')
    con.commit()
    con.close()

../daum_news_dataset/addDB\daum_news_20210101-20210131.csv
521638
../daum_news_dataset/addDB\daum_news_20210401-20210430.csv


C:\Users\kimminsung\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (5,6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


542043
../daum_news_dataset/addDB\daum_news_20210801-20210831.csv


C:\Users\kimminsung\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (1,2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


486489


# main

In [1]:
import glob
import pandas as pd
import re
import os
import sqlite3
import numpy as np
import csv

daumNews_dir = glob.glob('../glob_test2/*.csv')


def preprocessing_articles(data):
    contents_new = []
    for content in data['content']:
        # content = content.fillna("")
        content = str(content).split('\n')
        content = list(filter(lambda s: len(s) > 3, content))

        # 글 마지막에 항상 기자의 메일 주소가 추가되어 있다. 그 이후로 나오는 글은 불필요하므로 삭제함
        for i in range(1, len(content)):
            if '@' in content[-i]:
                break
        if i == len(content) - 1:
            i = 1
        content_prep = content[:-i]

            # 너무 짧거나, 광고, copyright, 사진설명, 메일 등 불필요한 단어가 포함되어 있을 경우 불필요한 문장으로 판단하고 삭제 처리
        content_prep = list(filter(lambda s: len(s) > 30 and \
                                             '▶' not in s and \
                                             '©' not in s and \
                                             '▲' not in s and \
                                             '사진=' not in s and \
                                             '사진제공=' not in s and \
                                             '@' not in s and \
                                             not re.findall("기자 *$", s) and \
                                             not re.findall("제공 *$", s) and \
                                             not re.findall("이하", s) and \
                                             not re.findall("무단전재", s) and \
                                             not re.findall("기자 *= *", s), content_prep))

        # 언론사명이 들어간 prefix 삭제
        content_prep = list(map(lambda s: re.sub("\[.*\]", "", s), content_prep))  # [스포츠서울]
        content_prep = list(map(lambda s: re.sub("\(.*\=.*\)", "", s), content_prep))  # (서울=뉴시스)
        content_prep = list(map(lambda s: re.sub("\【.*\=.*\】", "", s), content_prep))  # 【파이낸셜뉴스 포천=강근주 기자】
        # content_prep = list(map(lambda s: re.sub('[^A-Za-z가-힣]',' ', s), content_prep)) # 한글 제외 영문, 한자, 숫자, 특수문자 모두 제거
        content_prep = list(map(lambda s: re.sub('[^가-힣]', ' ', s), content_prep))  # 한글 제외 영문, 한자, 숫자, 특수문자 모두 제거

        contents_new.append('\n'.join(content_prep))
        # print('for 안',len(contents_new))
    # cnt =+ 1
    # print(cnt, contents_new)
    # print('for 밖',len(contents_new))
    data['news_content'] = contents_new
    return data


if __name__ == "__main__":
    con = sqlite3.connect("test.db")   # 데이터베이스 이름
    c = con.cursor()
    c.execute('DROP TABLE IF EXISTS `test_dummy`;')
    c.execute('''
            CREATE TABLE `test_dummy` (
                `news_id` INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
                `news_name` TEXT NOT NULL,
                `pubdate` TEXT NOT NULL,
                `url` TEXT NOT NULL,
                `news_content` TEXT NOT NULL
            );

    ''')
    full_dataset = pd.DataFrame()
    for daumNews in daumNews_dir:
        df = pd.read_csv(daumNews, names=['order', 'news_name', 'pubdate', 'url', 'content'])  # 컬럼이름 지정
        df = df.drop(['order'], axis=1)
        # print(df)
        data_prep = preprocessing_articles(df)
        full_dataset = pd.concat([full_dataset, data_prep], axis=0)
        # full_dataset.to_csv('test.csv', encoding='utf-8-sig')
        
        # print(data_prep)
        # full_dataset['news_content'] = full_dataset['news_content'].str.replace(' ', '')
        full_dataset['news_content'] = full_dataset['news_content'].str.replace('\n',' ')
        full_dataset['news_content'] = full_dataset['news_content'].str.strip()
        full_dataset['news_content'] = full_dataset['news_content'].replace('', np.nan)    # 빈셀이 있는 행을 삭제하기 위해 NaN으로 변경
        data_prep_drop = full_dataset.dropna(axis=0)
        data_prep_drop_dup = data_prep_drop.drop_duplicates(['news_content'], keep='first', inplace=False, ignore_index=True)   # 중복값 제거
        result = data_prep_drop_dup[['news_name', 'pubdate', 'url', 'news_content']]  # 비교를 위해 content(원본데이터) 추가, news_content가 전처리된 데이터
        # result.to_csv('test.csv', encoding='utf-8-sig')
    result.to_sql('test_dummy', con, if_exists='append', index=False)         # DB에 news_dummy 라는 테이블 이름으로 저장

    con.commit()
    con.close()

In [9]:
import pandas as pd
import re
import os
import sqlite3
import numpy as np

df = pd.read_csv('daum_news_test.csv', names=['order','news_name','pubdate','url', 'content'])   # 컬럼이름 지정
df.to_csv('daum_news_test_addCol.csv', encoding='utf-8-sig')

con = sqlite3.connect("news_test02.db")   # 데이터베이스 이름
c = con.cursor()

contents = []
contents_new = []

def preprocessing_articles(data):
    for content in data['content']:
        # print(content)
        
        # content = content.fillna("")
        content = str(content).split('\n')
        content = list(filter(lambda s: len(s) > 3, content))

        # 글 마지막에 항상 기자의 메일 주소가 추가되어 있다. 그 이후로 나오는 글은 불필요하므로 삭제함
        for i in range(1, len(content)):
            if '@' in content[-i]:
                break               
            if i == len(content)-1:
                i = 1
            content_prep = content[:-i]

        # 너무 짧거나, 광고, copyright, 사진설명, 메일 등 불필요한 단어가 포함되어 있을 경우 불필요한 문장으로 판단하고 삭제 처리
            content_prep = list(filter(lambda s: len(s) > 30 and \
                '▶' not in s and \
                    '©' not in s and \
                        '▲' not in s and \
                            '사진=' not in s and \
                                '사진제공=' not in s and \
                                    '@' not in s and \
                                        not re.findall("기자 *$", s) and \
                                            not re.findall("제공 *$", s) and \
                                                not re.findall("이하", s) and \
                                                    not re.findall("무단전재", s) and \
                                                        not re.findall("기자 *= *", s) , content_prep))

        # 언론사명이 들어간 prefix 삭제
        content_prep = list(map(lambda s: re.sub("\[.*\]", "", s), content_prep)) #[스포츠서울]
        content_prep = list(map(lambda s: re.sub("\(.*\=.*\)", "", s), content_prep)) #(서울=뉴시스)
        content_prep = list(map(lambda s: re.sub("\【.*\=.*\】", "", s), content_prep)) #【파이낸셜뉴스 포천=강근주 기자】
        content_prep = list(map(lambda s: re.sub('[^가-힣]',' ', s), content_prep)) # 한글 제외 영문, 한자, 숫자, 특수문자 모두 제거 

        contents.append('\n'.join(content))
        contents_new.append('\n'.join(content_prep))

    data['news_content'] = contents_new

    return data


if __name__ == "__main__":
        
    file = 'daum_news_test_addCol.csv'
    data = pd.read_csv(file)

    base_name, ext = os.path.splitext(os.path.basename(file))
    output_file = "{}_{}{}".format(base_name, "prep", ext)   # 파일이름을 위에서 저장한 파일이름에 prep을 붙여 생성

    data_prep = preprocessing_articles(data)
    data_prep['news_content'].replace('', np.nan, inplace=True)    # 빈셀이 있는 행을 삭제하기 위해 NaN으로 변경
    data_prep_drop = data_prep.dropna(axis=0)
    data_prep_drop_dup = data_prep_drop.drop_duplicates(['news_content'], keep='first', inplace=False, ignore_index=True)   # 중복값 제거
    result = data_prep_drop_dup[['news_name', 'pubdate', 'url', 'content', 'news_content']]  # 비교를 위해 content(원본데이터) 추가, news_content가 전처리된 데이터
    result.to_sql('news_dummy', con, if_exists='append')         # DB에 news_dummy 라는 테이블 이름으로 저장
    con.commit()
    con.close()
    result.to_csv(output_file, index=None, encoding='utf-8-sig')   # 테이터 확인을 위해 csv파일로 저장

In [49]:
import glob
import pandas as pd
import re
import os
import sqlite3
import numpy as np


daumNews_dir = glob.glob('./glob_test/*.csv')


contents_new = []
cnt = 0
def preprocessing_articles(data):
    for content in data['content']:
        # print(content)

        # content = content.fillna("")
        content = str(content).split('\n')
        content = list(filter(lambda s: len(s) > 3, content))

        # 글 마지막에 항상 기자의 메일 주소가 추가되어 있다. 그 이후로 나오는 글은 불필요하므로 삭제함
        for i in range(1, len(content)):
            if '@' in content[-i]:
                break               
            if i == len(content)-1:
                i = 1
            content_prep = content[:-i]

        # 너무 짧거나, 광고, copyright, 사진설명, 메일 등 불필요한 단어가 포함되어 있을 경우 불필요한 문장으로 판단하고 삭제 처리
            content_prep = list(filter(lambda s: len(s) > 30 and \
                '▶' not in s and \
                    '©' not in s and \
                        '▲' not in s and \
                            '사진=' not in s and \
                                '사진제공=' not in s and \
                                    '@' not in s and \
                                        not re.findall("기자 *$", s) and \
                                            not re.findall("제공 *$", s) and \
                                                not re.findall("이하", s) and \
                                                    not re.findall("무단전재", s) and \
                                                        not re.findall("기자 *= *", s) , content_prep))

        # 언론사명이 들어간 prefix 삭제
        content_prep = list(map(lambda s: re.sub("\[.*\]", "", s), content_prep)) #[스포츠서울]
        content_prep = list(map(lambda s: re.sub("\(.*\=.*\)", "", s), content_prep)) #(서울=뉴시스)
        content_prep = list(map(lambda s: re.sub("\【.*\=.*\】", "", s), content_prep)) #【파이낸셜뉴스 포천=강근주 기자】
        # content_prep = list(map(lambda s: re.sub('[^A-Za-z가-힣]',' ', s), content_prep)) # 한글 제외 영문, 한자, 숫자, 특수문자 모두 제거 
        content_prep = list(map(lambda s: re.sub('[^가-힣]',' ', s), content_prep)) # 한글 제외 영문, 한자, 숫자, 특수문자 모두 제거 


        contents_new.append('\n'.join(content_prep))
        # print('for 안',len(contents_new))
    # cnt =+ 1
    # print(cnt, contents_new)
    # print('for 밖',len(contents_new))
    data['news_content'] = contents_new
    return data


if __name__ == "__main__":
    # con = sqlite3.connect("glob_test.db")   # 데이터베이스 이름
    # # c = con.cursor()
    # c = con.cursor()
    # c.execute('DROP TABLE IF EXISTS `news_dummy`;')
    # c.execute('''           
    #         CREATE TABLE `news_dummy` (
    #             `news_id` INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
    #             `news_name` TEXT NOT NULL,
    #             `pubdate` TEXT NOT NULL,
    #             `url` TEXT NOT NULL,
    #             `news_content` TEXT NOT NULL
    #         );

    # ''')

    for daumNews in daumNews_dir:
        df = pd.read_csv(daumNews, names=['order','news_name','pubdate','url', 'content'])   # 컬럼이름 지정
        df = df.drop(['order'], axis=1)
        # print(df)
        data_prep = preprocessing_articles(df)
        # print(data_prep)
        # data_prep['news_content'] = data_prep['news_content'].str.replace(' ', '')
        # data_prep['news_content'] = data_prep['news_content'].str.replace('\n',' ')
        # data_prep['news_content'] = data_prep['news_content'].str.strip()
        # data_prep['news_content'] = data_prep['news_content'].replace('', np.nan)    # 빈셀이 있는 행을 삭제하기 위해 NaN으로 변경
        # data_prep_drop = data_prep.dropna(axis=0)
        # data_prep_drop_dup = data_prep_drop.drop_duplicates(['news_content'], keep='first', inplace=False, ignore_index=True)   # 중복값 제거
        # result = data_prep_drop_dup[['news_name', 'pubdate', 'url', 'news_content']]  # 비교를 위해 content(원본데이터) 추가, news_content가 전처리된 데이터
        # print(result)
        # c = con.cursor()
        # result.to_sql('news_dummy', con, if_exists='append', index=False)         # DB에 news_dummy 라는 테이블 이름으로 저장
    
    # con.commit()
    # con.close()


ValueError: Length of values (30) does not match length of index (15)

In [5]:
data_prep

,news_name,pubdate,url,content,news_content
0,BRITAIN FORMULA ONE GRAND PRIX,2020. 08. 01. 23:59,https://v.daum.net/v/20200801235958458,British Formula One Grand Prix \n\n\nepa085790...,NaN
1,GERMANY TRAVEL PANDEMIC CORONAVIRUS COVID19,2020. 08. 01. 23:59,https://v.daum.net/v/20200801235935457,German Federal Foreign Office issues travel wa...,NaN
2,BRITAIN FORMULA ONE GRAND PRIX,2020. 08. 01. 23:59,https://v.daum.net/v/20200801235932456,British Formula One Grand Prix \n\n\nepa085790...,NaN
3,수도권 등 '물폭탄'에 사망 1명·곳곳 침수..내일도 최대 80mm(종합),2020. 08. 01. 23:59,https://v.daum.net/v/20200801235926455,경기 파주시 이날 하루만 최대 124mm 폭우서울 도림천 80대 남성 구조..이송 ...,경기 파주시 이날 하루만 최대 폭우서울 도림천 대 남성 구조 이송 ...
4,GERMANY TRAVEL PANDEMIC CORONAVIRUS COVID19,2020. 08. 01. 23:59,https://v.daum.net/v/20200801235925454,German Federal Foreign Office issues travel wa...,NaN
...,...,...,...,...,...
705211,거리두기 2.5단계 속 정기국회 개막준비..공공의대·공수처·4차추경 쟁점,2020. 08. 31. 00:00,https://v.daum.net/v/20200831000003423,"9월 1일 정기국회 개막, 100일 대장정 돌입4차 추경안과 2차 재난지원금 첫 쟁...",월 일 정기국회 개막 일 대장정 돌입 차 추경안과 차 재난지원금 첫 쟁정...
705212,"""아 옛날이여"" 해태제과, 고향만두·허니버터칩 영광 되찾을까",2020. 08. 31. 00:00,https://v.daum.net/v/20200831000003422,'고향만두' 출시 이후 지난 2012년까지 줄곧 국내 냉동 만두시장 점유율 1위를 ...,지난 년까지 무려 여 년간 냉동 만두시장 위를 지키던 해태제과가 올해 ...
705213,"김도훈의 놀라움과 감동 ""그래서 이청용, 이청용 하나봐요""",2020. 08. 31. 00:00,https://v.daum.net/v/20200831000003421,▲ 즐거운 울산 현대 김도훈 감독 ⓒ한국프로축구연맹\n \n\n[...,그래서 이청용 이청용 하는 것 같아요 김도훈 울산 현대 감독도 쌍용 이청...
705214,[인사] 병무청,2020. 08. 31. 00:00,https://v.daum.net/v/20200831000003420,(서울=뉴스1) = ◆병무청 <승진>▷서기관 Δ병역자원국 황주하 Δ사회복무국 김형숙...,병무청 승진 서기관 병역자원국 황주하 사회복무국 김형숙 사회복무국 송민선


In [6]:
result

,news_name,pubdate,url,news_content
0,수도권 등 '물폭탄'에 사망 1명·곳곳 침수..내일도 최대 80mm(종합),2020. 08. 01. 23:59,https://v.daum.net/v/20200801235926455,경기 파주시 이날 하루만 최대 폭우서울 도림천 대 남성 구조 이송 ...
1,"'트레인' 윤시윤, 평행세계 자신 전화에 패닉 ""차엽은 공범"" (종합)",2020. 08. 01. 23:55,https://v.daum.net/v/20200801235556443,월 일 방송된 토일드라마 트레인 회 극본 박가연 연출 류승진 이승훈 ...
2,하남 미사2동 취약계층 선풍기 55대전달,2020. 08. 01. 23:55,https://v.daum.net/v/20200801235522442,하남시 미사 동행정복지센터는 월 일 미사 동 지역사회보장협의체와 하일위생환경이...
3,"'온앤오프' 소유 ""다이어트 중, 달걀+치즈 라면+소맥 먹고 싶어"" [TV캡처]",2020. 08. 01. 23:55,https://v.daum.net/v/20200801235511441,온앤오프 소유가 뮤직비디오 촬영으로 인한 다이어트가 끝난 뒤 먹고 싶은 음식을 밝...
4,"더불어민주당 홍성국 의원, 가상화폐 돈세탁 감독법 대표발의",2020. 08. 01. 23:55,https://v.daum.net/v/20200801235505440,빗썸 등 가상자산사업자도 금융당국에 자금세탁방지 업무 감독 받아야 ...
...,...,...,...,...
530762,거리두기 2.5단계 속 정기국회 개막준비..공공의대·공수처·4차추경 쟁점,2020. 08. 31. 00:00,https://v.daum.net/v/20200831000003423,월 일 정기국회 개막 일 대장정 돌입 차 추경안과 차 재난지원금 첫 쟁정...
530763,"""아 옛날이여"" 해태제과, 고향만두·허니버터칩 영광 되찾을까",2020. 08. 31. 00:00,https://v.daum.net/v/20200831000003422,지난 년까지 무려 여 년간 냉동 만두시장 위를 지키던 해태제과가 올해 ...
530764,"김도훈의 놀라움과 감동 ""그래서 이청용, 이청용 하나봐요""",2020. 08. 31. 00:00,https://v.daum.net/v/20200831000003421,그래서 이청용 이청용 하는 것 같아요 김도훈 울산 현대 감독도 쌍용 이청...
530765,[인사] 병무청,2020. 08. 31. 00:00,https://v.daum.net/v/20200831000003420,병무청 승진 서기관 병역자원국 황주하 사회복무국 김형숙 사회복무국 송민선


In [7]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530767 entries, 0 to 530766
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   news_name     530767 non-null  object
 1   pubdate       530767 non-null  object
 2   url           530767 non-null  object
 3   news_content  530767 non-null  object
dtypes: object(4)
memory usage: 16.2+ MB


In [8]:
result['news_content']

0         경기 파주시 이날 하루만 최대       폭우서울 도림천   대 남성 구조  이송 ...
1         월  일 방송된     토일드라마  트레인   회 극본 박가연 연출 류승진 이승훈 ...
2         하남시 미사 동행정복지센터는  월  일 미사 동 지역사회보장협의체와  하일위생환경이...
3         온앤오프  소유가 뮤직비디오 촬영으로 인한 다이어트가 끝난 뒤 먹고 싶은 음식을 밝...
4         빗썸 등 가상자산사업자도 금융당국에 자금세탁방지 업무 감독 받아야          ...
                                ...                        
530762    월  일 정기국회 개막     일 대장정 돌입 차 추경안과  차 재난지원금 첫 쟁정...
530763    지난     년까지 무려   여 년간 냉동 만두시장  위를 지키던 해태제과가 올해 ...
530764    그래서 이청용  이청용 하는 것 같아요    김도훈 울산 현대 감독도  쌍용  이청...
530765        병무청  승진  서기관  병역자원국 황주하  사회복무국 김형숙  사회복무국 송민선
530766    베이지 컬러만큼 톤온톤 스타일링을 하기 좋은 컬러가 있  베이지 컬러만큼 톤온톤 스...
Name: news_content, Length: 530767, dtype: object

In [48]:
result

,news_name,pubdate,url,news_content,remove_n
0,BRITAIN FORMULA ONE GRAND PRIX,2020. 08. 01. 23:59,https://v.daum.net/v/20200801235958458,British Formula One Grand Prix \nepa B...,British Formula One Grand Prix epa Bri...
1,GERMANY TRAVEL PANDEMIC CORONAVIRUS COVID19,2020. 08. 01. 23:59,https://v.daum.net/v/20200801235935457,German Federal Foreign Office issues travel wa...,German Federal Foreign Office issues travel wa...
2,수도권 등 '물폭탄'에 사망 1명·곳곳 침수..내일도 최대 80mm(종합),2020. 08. 01. 23:59,https://v.daum.net/v/20200801235926455,경기 파주시 이날 하루만 최대 mm 폭우서울 도림천 대 남성 구조 이송 ...,경기 파주시 이날 하루만 최대 mm 폭우서울 도림천 대 남성 구조 이송 ...
3,GERMANY TRAVEL PANDEMIC CORONAVIRUS COVID19,2020. 08. 01. 23:59,https://v.daum.net/v/20200801235925454,German Federal Foreign Office issues travel wa...,German Federal Foreign Office issues travel wa...
4,California Wildfires,2020. 08. 01. 23:57,https://v.daum.net/v/20200801235718444,A firefighter battles a brush fire at the Appl...,A firefighter battles a brush fire at the Appl...
...,...,...,...,...,...
567592,거리두기 2.5단계 속 정기국회 개막준비..공공의대·공수처·4차추경 쟁점,2020. 08. 31. 00:00,https://v.daum.net/v/20200831000003423,월 일 정기국회 개막 일 대장정 돌입 차 추경안과 차 재난지원금 첫 쟁...,월 일 정기국회 개막 일 대장정 돌입 차 추경안과 차 재난지원금 첫 쟁...
567593,"""아 옛날이여"" 해태제과, 고향만두·허니버터칩 영광 되찾을까",2020. 08. 31. 00:00,https://v.daum.net/v/20200831000003422,지난 년까지 무려 여 년간 냉동 만두시장 위를 지키던 해태제과가 올해...,지난 년까지 무려 여 년간 냉동 만두시장 위를 지키던 해태제과가 올해...
567594,"김도훈의 놀라움과 감동 ""그래서 이청용, 이청용 하나봐요""",2020. 08. 31. 00:00,https://v.daum.net/v/20200831000003421,그래서 이청용 이청용 하는 것 같아요 \n김도훈 울산 현대 감독도 쌍용 ...,그래서 이청용 이청용 하는 것 같아요 김도훈 울산 현대 감독도 쌍용 이...
567595,[인사] 병무청,2020. 08. 31. 00:00,https://v.daum.net/v/20200831000003420,병무청 승진 서기관 병역자원국 황주하 사회복무국 김형숙 사회복무국 송민선,병무청 승진 서기관 병역자원국 황주하 사회복무국 김형숙 사회복무국 송민선
